# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 3 2023 12:42PM,256624,12,U1207,"Uniderm USA, Inc",Released
1,Feb 3 2023 12:40PM,256623,12,7547722,"NBTY Global, Inc.",Released
2,Feb 3 2023 12:35PM,256621,16,9111824,Sartorius Bioprocess Solutions,Released
3,Feb 3 2023 12:30PM,256620,10,0086354605,ISDIN Corporation,Released
4,Feb 3 2023 12:30PM,256620,10,0086354606,ISDIN Corporation,Released
5,Feb 3 2023 12:30PM,256620,10,0086354602,ISDIN Corporation,Released
6,Feb 3 2023 12:30PM,256620,10,0086354603,ISDIN Corporation,Released
7,Feb 3 2023 12:30PM,256620,10,0086354607,ISDIN Corporation,Released
8,Feb 3 2023 12:30PM,256620,10,0086354604,ISDIN Corporation,Released
9,Feb 3 2023 12:30PM,256620,10,0086354627,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,256619,Released,2
39,256620,Released,21
40,256621,Released,1
41,256623,Released,1
42,256624,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256619,NaN,NaN,2.0
256620,NaN,NaN,21.0
256621,NaN,NaN,1.0
256623,NaN,NaN,1.0
256624,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256551,0.0,0.0,3.0
256556,0.0,1.0,0.0
256580,0.0,0.0,1.0
256583,2.0,13.0,0.0
256585,1.0,3.0,18.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256551,0,0,3
256556,0,1,0
256580,0,0,1
256583,2,13,0
256585,1,3,18


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256551,0,0,3
1,256556,0,1,0
2,256580,0,0,1
3,256583,2,13,0
4,256585,1,3,18


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256551,,,3
1,256556,,1,
2,256580,,,1
3,256583,2,13,
4,256585,1,3,18


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 3 2023 12:42PM,256624,12,"Uniderm USA, Inc"
1,Feb 3 2023 12:40PM,256623,12,"NBTY Global, Inc."
2,Feb 3 2023 12:35PM,256621,16,Sartorius Bioprocess Solutions
3,Feb 3 2023 12:30PM,256620,10,ISDIN Corporation
24,Feb 3 2023 12:28PM,256619,10,Emerginnova
26,Feb 3 2023 12:25PM,256618,10,ISDIN Corporation
31,Feb 3 2023 11:51AM,256615,10,ISDIN Corporation
36,Feb 3 2023 11:19AM,256614,19,"AdvaGen Pharma, Ltd"
57,Feb 3 2023 11:09AM,256613,19,"AdvaGen Pharma, Ltd"
58,Feb 3 2023 10:41AM,256610,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 3 2023 12:42PM,256624,12,"Uniderm USA, Inc",,,1
1,Feb 3 2023 12:40PM,256623,12,"NBTY Global, Inc.",,,1
2,Feb 3 2023 12:35PM,256621,16,Sartorius Bioprocess Solutions,,,1
3,Feb 3 2023 12:30PM,256620,10,ISDIN Corporation,,,21
4,Feb 3 2023 12:28PM,256619,10,Emerginnova,,,2
5,Feb 3 2023 12:25PM,256618,10,ISDIN Corporation,,,5
6,Feb 3 2023 11:51AM,256615,10,ISDIN Corporation,,,5
7,Feb 3 2023 11:19AM,256614,19,"AdvaGen Pharma, Ltd",,14,7
8,Feb 3 2023 11:09AM,256613,19,"AdvaGen Pharma, Ltd",,,1
9,Feb 3 2023 10:41AM,256610,19,"AdvaGen Pharma, Ltd",,2,15


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 3 2023 12:42PM,256624,12,"Uniderm USA, Inc",1,,
1,Feb 3 2023 12:40PM,256623,12,"NBTY Global, Inc.",1,,
2,Feb 3 2023 12:35PM,256621,16,Sartorius Bioprocess Solutions,1,,
3,Feb 3 2023 12:30PM,256620,10,ISDIN Corporation,21,,
4,Feb 3 2023 12:28PM,256619,10,Emerginnova,2,,
5,Feb 3 2023 12:25PM,256618,10,ISDIN Corporation,5,,
6,Feb 3 2023 11:51AM,256615,10,ISDIN Corporation,5,,
7,Feb 3 2023 11:19AM,256614,19,"AdvaGen Pharma, Ltd",7,14,
8,Feb 3 2023 11:09AM,256613,19,"AdvaGen Pharma, Ltd",1,,
9,Feb 3 2023 10:41AM,256610,19,"AdvaGen Pharma, Ltd",15,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 3 2023 12:42PM,256624,12,"Uniderm USA, Inc",1,,
1,Feb 3 2023 12:40PM,256623,12,"NBTY Global, Inc.",1,,
2,Feb 3 2023 12:35PM,256621,16,Sartorius Bioprocess Solutions,1,,
3,Feb 3 2023 12:30PM,256620,10,ISDIN Corporation,21,,
4,Feb 3 2023 12:28PM,256619,10,Emerginnova,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 3 2023 12:42PM,256624,12,"Uniderm USA, Inc",1.0,NaN,NaN
1,Feb 3 2023 12:40PM,256623,12,"NBTY Global, Inc.",1.0,NaN,NaN
2,Feb 3 2023 12:35PM,256621,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
3,Feb 3 2023 12:30PM,256620,10,ISDIN Corporation,21.0,NaN,NaN
4,Feb 3 2023 12:28PM,256619,10,Emerginnova,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 3 2023 12:42PM,256624,12,"Uniderm USA, Inc",1.0,0.0,0.0
1,Feb 3 2023 12:40PM,256623,12,"NBTY Global, Inc.",1.0,0.0,0.0
2,Feb 3 2023 12:35PM,256621,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
3,Feb 3 2023 12:30PM,256620,10,ISDIN Corporation,21.0,0.0,0.0
4,Feb 3 2023 12:28PM,256619,10,Emerginnova,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2565977,64.0,15.0,1.0
12,513247,2.0,0.0,0.0
15,1539587,22.0,37.0,9.0
16,513207,1.0,1.0,0.0
19,1026420,23.0,29.0,2.0
21,769800,3.0,0.0,0.0
22,513187,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2565977,64.0,15.0,1.0
1,12,513247,2.0,0.0,0.0
2,15,1539587,22.0,37.0,9.0
3,16,513207,1.0,1.0,0.0
4,19,1026420,23.0,29.0,2.0
5,21,769800,3.0,0.0,0.0
6,22,513187,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,64.0,15.0,1.0
1,12,2.0,0.0,0.0
2,15,22.0,37.0,9.0
3,16,1.0,1.0,0.0
4,19,23.0,29.0,2.0
5,21,3.0,0.0,0.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,64.0
1,12,Released,2.0
2,15,Released,22.0
3,16,Released,1.0
4,19,Released,23.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Completed,1.0,0.0,9.0,0.0,2.0,0.0,0.0
Executing,15.0,0.0,37.0,1.0,29.0,0.0,0.0
Released,64.0,2.0,22.0,1.0,23.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Completed,1.0,0.0,9.0,0.0,2.0,0.0,0.0
1,Executing,15.0,0.0,37.0,1.0,29.0,0.0,0.0
2,Released,64.0,2.0,22.0,1.0,23.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Completed,1.0,0.0,9.0,0.0,2.0,0.0,0.0
1,Executing,15.0,0.0,37.0,1.0,29.0,0.0,0.0
2,Released,64.0,2.0,22.0,1.0,23.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()